Importation des librairies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az
import scipy.stats as st
import pytensor.tensor as tt

Valeurs assignées et mesurées

In [ ]:
# Valeurs assignées
Assigned_Values = pd.DataFrame({
    "Matériau": ["DMR 486b", "DMR 274g", "SRM 1549a", "DMR 82c",
                 "GBW10037", "SRM 1869", "SRM 1849a", "GBW(E)100227",
                 "108 02 003"],
    "X": [4.51, 5.52, 5.91, 8.83, 39.8, 60.6, 65.3, 97, 108],
    "uX": [0.11, 0.13, 0.195, 0.205, 1.35, 0.65, 2.8, 2.00, 5.00],
    "Laboratoire": ['CENAM', 'CENAM', 'NIST', 'CENAM', 'NIM', 'KRISS',
                    'NIM', 'NIST', 'NIST']})

# Valeurs mesurées
Measured_Values = np.array([
    4.449, 4.433, 4.083, 4.554, 4.646, 4.210,   # DMR-486b (campagne 1)
    4.684, 4.806, 4.491, 4.501, 4.486, 4.471,            # (campagne 2)
    4.637, 4.528, 4.692, 4.540, 4.501, 4.620,            # (campagne 3)
    5.433, 5.334, 5.417, 5.459, 5.445, 5.488,   # DMR-274g
    5.598, 5.352, 5.893, 5.692, 5.282, 5.450,
    5.575, 5.436, 5.609, 5.586, 5.189, 5.554,
    6.031, 5.827, 6.024, 6.027, 5.539, 5.721,   # SMR 1549a
    5.592, 5.986, 6.221, 5.981, 5.096, 4.983,
    5.369, 5.394, 5.973, 5.909, 5.556, 5.762,
    8.724, 9.207, 8.916, 8.853, 8.952, 8.636,   # DMR-82c
    9.189, 9.117, 9.179, 8.553, 9.043, 8.979,
    8.609, 8.539, 8.549, 8.421, 8.621, 8.166,
    36.49, 36.94, 37.98, 36.81, 37.50, 37.45,   # GBW10037
    35.01, 35.56, 37.79, 36.14, 35.29, 36.92,
    39.25, 39.49, 37.58, 37.20, 39.14, 37.95,
    60.53, 59.99, 61.79, 61.63, 58.69, 60.85,   # SRM 1869
    61.81, 60.23, 60.11, 61.07, 61.43, 59.44,
    59.16, 59.99, 59.68, 59.58, 60.83, 58.72,
    65.32, 64.56, 65.48, 65.25, 63.64, 64.71,   # SRM 1849a
    61.48, 63.33, 63.02, 63.49, 65.14, 65.15,
    64.66, 64.80, 64.42, 63.86, 63.82, 63.54,
    98.59, 96.57, 96.76, 96.83, 100.47, 100.75, # GBW(E)10027
    98.84, 98.59, 96.48, 94.42, 95.75, 97.97,
    100.27, 96.67, 97.02, 97.21, 100.69, 98.80,
    100.10, 98.51, 99.05, 101.62, 100.24, 100.30,   # 108-02-003
    100.33, 100.29, 101.97, 99.54, 101.29, 101.78,
    99.89, 99.20, 98.11, 98.74, 100.74, 99.48
]).reshape(-1, 6)

Tableaux de données principal

In [ ]:
materiaux = np.repeat(Assigned_Values["Matériau"].values, 3)
campagnes = np.tile([1, 2, 3], 9)

# Tableau de données format long
df = pd.DataFrame({
    "Matériau": np.repeat(materiaux, 6),
    "Campagne": np.repeat(campagnes, 6),
    "Aliquot": np.tile(np.repeat(np.arange(1, 4), 2), len(materiaux)),
    "Y": Measured_Values.flatten()  # by default to flatten row by row
}).merge(Assigned_Values, on="Matériau", how="left")

display(df)

Global display

In [ ]:
sns.set_theme(style="whitegrid", context="talk")
plt.figure(figsize=(14, 9))

# Error bars
for _, row in df.iterrows():
    plt.errorbar(row["X"], row["Y"], xerr=row["uX"], fmt='none',
                 ecolor='lightgray', alpha=0.4, zorder=0)

plot = sns.scatterplot(data=df, x="X", y="Y", hue="Matériau",
                       palette="colorblind", s=40, alpha=0.85)

# Line y = x
x_min, x_max = df["X"].min(), df["X"].max()
plt.plot([x_min, x_max], [x_min, x_max], color="lightgray",
         linestyle="--", linewidth=2, zorder=0, label="y = x")

# Legend
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1.05, 1),
           loc="upper left", title="Matériau")

plt.xlabel("Valeurs assignées (X)")
plt.ylabel("valeurs mesurées (Y)")
plt.tight_layout()
plt.show()

Local displays

In [ ]:
# Function to distribute points along the uncertainty bar for each
# material
def x_disp(df):
    df = df.copy()
    for mat in df["Matériau"].unique():
        mask = df["Matériau"] == mat
        n = mask.sum()
        x_central = df.loc[mask, "X"].iloc[0]
        incert = df.loc[mask, "uX"].iloc[0]
        margin = 0.15
        left = x_central - incert * (1 - margin)
        right = x_central + incert * (1 - margin)
        offsets = np.linspace(left, right, n)
        df.loc[mask, "X_disp"] = offsets
    return df

df_disp = x_disp(df)

# Plotting function for each facet
def plot_local(data, **kwargs):
# **kwargs receives additional keyword
# arguments from map_dataframe
    ax = plt.gca()
    ax.grid(color="lightgray", linestyle="--", linewidth=0.7, alpha=0.5)
    for _, row in data.iterrows():
        ax.errorbar(row["X"], row["Y"], xerr=row["uX"], fmt='none',
                    ecolor='gray', alpha=0.3, zorder=0)
    sns.scatterplot(data=data, x="X_disp", y="Y", hue="Campagne",
                    style="Aliquot", palette="colorblind", s=80, ax=ax,
                    alpha=0.95, legend=False)

    # Graph layout partitioning
    x0 = data["X"].iloc[0]
    incert = data["uX"].iloc[0]
    ax.axvline(x0, color="black", linestyle=":", alpha=0.4, zorder=0)
    ax.set_xlim(x0 - incert, x0 + incert)

# Facet grid
g = sns.FacetGrid(df_disp, col="Matériau", col_wrap=3, height=5,
                  sharex=False, sharey=False)
g.map_dataframe(plot_local)
g.set_titles("{col_name}")
g.set_axis_labels("X", "Y")

# Legend
plt.figure(figsize=(10, 6))
legend_plot = sns.scatterplot(data=df_disp, x="X_disp", y="Y",
                              hue="Campagne", style="Aliquot",
                              palette="colorblind")
handles, labels = legend_plot.get_legend_handles_labels()
plt.close()
# close the temporary figure after extracting legend
# elements
g.figure.legend(handles, labels, bbox_to_anchor=(1.02, 0.5),
                loc="center left")

plt.tight_layout()
plt.show()

**Uncertainty calculations**

Combined standard uncertainty:

$$
u_{DOE} = \sqrt{\frac{N_a \times N_r \times \sigma_c^2 + N_r \times \sigma_a^2 + \sigma_r^2}{N_c \times N_a \times N_r}}
$$

Expanded uncertainty at 95% confidence:

$$
U_{95} = \sqrt{(t \times u_{DOE})^2 + (2 \times \sigma_{char})^2}
$$

Final standard uncertainty:

$$
u_{inf} = \frac{U_{95}}{2}
$$

Symbols:

- $N_a$: number of campaigns
- $N_r$: number of aliquots
- $N_c$: number of replicates
- $\sigma_c$: campaign effect
- $\sigma_a$: aliquot effect
- $\sigma_r$: repeatability
- $t$:  coverage factor (Student’s t-distribution)
- $\sigma_{char}$: characterization uncertainty

In [ ]:
N_a, N_c, N_r = 3, 3, 2

sigma_c  = np.array([0, 0, 0, 0.16701, 0, 0, 0, 0.7087, 0])
sigma_a = np.array([0, 0, 0.15444, 0, 0.3858, 0.66436, 0, 0, 0])
sigma_r = np.array([0.16724, 0.15921, 0.13728, 0.20217, 0.5787, 1.28964,
                    1.0978, 0.8952, 1.0251])

u_DOE = np.sqrt((N_a * N_r * sigma_c**2 + N_r * sigma_a**2 + sigma_r**2)
 / (N_c * N_a * N_r))
t = np.array([2.1, 2.1, 2.3, 4.3, 2.3, 2.3, 2.1, 4.3, 2.1])
sigma_char = np.array([0.090852, 0.1098, 0.12012, 0.180195, 1.286,
                       2.01262, 1.996, 0.6714, 1.08542])

U_95 = np.sqrt((t * u_DOE)**2 + (2 * sigma_char)**2)

u_inf = U_95 / 2

# Assigning the calculated uncertainties to the data frame
df["uY"] = df["Matériau"].map(dict(zip(Assigned_Values["Matériau"],
                                       u_inf)))

Bayesian model

In [ ]:
# Dimensions
n0, n1, n2, n3 = 9, 3, 3, 2

# Données observées
X_obs, uX = Assigned_Values["X"].values, Assigned_Values["uX"].values
mat_order = Assigned_Values["Matériau"].values
Y_obs = df["Y"].groupby(df["Matériau"]).mean().reindex(mat_order).values
uY = df["uY"].groupby(df["Matériau"]).mean().reindex(mat_order).values
Rdat = Measured_Values.reshape(n0, n1, n2, n3)
uRda1 = np.array([0.09, 0.11, 0.12, 0.18, 0.66, 1.10, 1.30, 2.01, 2.03])

# Model
with pm.Model() as model:
    uRda1_data = pm.Data('uRda1', uRda1)
    uX_data = pm.Data('uX', uX)
    uY_data = pm.Data('uY', uY)

    # Paramètres de régression
    a = pm.Normal('a', mu = 0, sigma = 1 / np.sqrt(1e-5))
    b = pm.Normal('b', mu = 1, sigma = 1 / np.sqrt(1e-5))

    # Paramètres instrumentaux
    sigma_mat = pm.HalfNormal('sigma_mat', sigma = 1)
    smthd = pm.Deterministic('smthd', 100 / pm.math.sqrt(sigma_mat))

    prept = pm.Deterministic('prept', sigma_mat / (uX_data ** 2))
    srept = pm.Deterministic('srept', 1 / pm.math.sqrt(prept))

    # Modèle mesurand
    smodel = pm.Normal('smodel', mu=0, sigma = uRda1_data, shape = n0)

    # Variables certifiées Vtru et observations Vda1
    Vtru = pm.Normal('Vtru', mu = 0, sigma = 1 / np.sqrt(1e-5),
                     shape = n0)
    Vda1_obs = pm.Normal('Vda1_obs', mu = Vtru, sigma = uX_data,
                         observed = X_obs)

    # Variables Vhat et observations Vda2
    Vhat = pm.Normal('Vhat', mu = 0, sigma = 1 / np.sqrt(1e-5),
                     shape = n0)
    Vda2_obs = pm.Normal('Vda2_obs', mu = Vhat, sigma = uY_data,
                         observed = Y_obs)

    # Régression prédictive
    alpha = pm.Deterministic('alpha', a + b * Vhat)
    Rhatuc = pm.Deterministic('Rhatuc', alpha + smodel)
    Vhatuc = pm.Deterministic('Vhatuc', (Rhatuc - a) / b)

    # Biais hiérarchiques beta, gamma
    sigma_cam = pm.HalfNormal('sigma_cam', sigma = 1, shape = (n0, n1))
    beta = pm.Normal('beta', mu = alpha[:, None], sigma = 1 /
                     pm.math.sqrt(prept[:, None]), shape = (n0, n1))

    sigma_ali = pm.HalfNormal('sigma_ali', sigma = 1,
                              shape = (n0, n1, n2))
    gamma = pm.Normal('gamma', mu = beta[:, :, None], sigma = 1 /
                      pm.math.sqrt(sigma_cam[:, :, None]),
                      shape = (n0, n1, n2))

    # Observations répétées Rdat
    Rdat_obs = pm.Normal('Rdat_obs', mu = tt.repeat(gamma[:, :, :, None],
                                                    n3, axis=3),
                          sigma = 1 / pm.math.sqrt(tt.repeat(
                              sigma_ali[:, :, :, None], n3, axis=3)),
                          observed = Rdat)

    # Calcul du degré d'équivalence
    doe = pm.Deterministic('doe', 200 * (Vtru - Vhatuc)
    / (Vtru + Vhatuc))

    initvals = {"sigma_mat": 1, "sigma_cam": np.ones((n0, n1)),
                "sigma_ali": np.ones((n0, n1, n2))}

    trace = pm.sample(draws = 4000, tune = 12000, target_accept = 0.99,
                      nuts_sampler = 'numpyro', random_seed = 23,
                      initvals = initvals)

# Directed Acyclic Graph
pm.model_to_graphviz(model)

Résumé brut de la trace

In [ ]:
summary = az.summary(trace, hdi_prob = 0.95)
display(summary)

Résumé poli de la trace

In [ ]:
summary = az.summary(trace, var_names = ["alpha", "beta", "gamma",
                                         "sigma_mat", "sigma_cam",
                                         "sigma_ali"], hdi_prob = 0.95)
summary = summary[["mean", "sd", "hdi_2.5%", "hdi_97.5%"]]

# Ajout d'alpha_mean
alpha = az.extract(trace, var_names = ["alpha"])
alpha_mean = alpha.mean(dim = "alpha_dim_0")
row_alpha_mean = {"mean": float(alpha_mean.mean()),
                  "sd": float(alpha_mean.std()),
                  "hdi_2.5%": np.percentile(alpha_mean, 2.5),
                  "hdi_97.5%": np.percentile(alpha_mean, 97.5)}
extra = pd.DataFrame([row_alpha_mean], index = ["alpha_mean"])
summary = pd.concat([summary, extra])

# Ajout de beta_mean
beta = az.extract(trace, var_names = ["beta"])
beta_mean = beta.mean(dim = "beta_dim_1")
beta_mean_stats = []
for i in range(beta_mean.sizes["beta_dim_0"]):
    arr = beta_mean.sel(beta_dim_0 = i).values.flatten()
    beta_mean_stats.append({
        "mean": arr.mean(),
        "sd": arr.std(),
        "hdi_2.5%": np.percentile(arr, 2.5),
        "hdi_97.5%": np.percentile(arr, 97.5)
        })
extra = pd.DataFrame(
    beta_mean_stats,
    index = [f"beta_{i}_mean" for i in range(len(
        beta_mean_stats))])
summary = pd.concat([summary, extra])

# Ajout de gamma_mean
gamma = az.extract(trace, var_names = ["gamma"])
gamma_mean = gamma.mean(dim = "gamma_dim_2")
gamma_mean_stats = []
for i in range(gamma_mean.sizes["gamma_dim_0"]):
    for j in range(gamma_mean.sizes["gamma_dim_1"]):
        arr = gamma_mean.sel(
            gamma_dim_0 = i, gamma_dim_1 = j).values.flatten()
        gamma_mean_stats.append({
            "mean": arr.mean(),
            "sd": arr.std(),
            "hdi_2.5%": np.percentile(arr, 2.5),
            "hdi_97.5%": np.percentile(arr, 97.5)
        })

extra = pd.DataFrame(
    gamma_mean_stats,
    index=[f"gamma_{i}_{j}_mean" for i in range(
        gamma_mean.sizes["gamma_dim_0"])
           for j in range(gamma_mean.sizes["gamma_dim_1"])]
)
summary = pd.concat([summary, extra])

# Ajout de sigma_cam
sigma_cam = az.extract(trace, var_names = ["sigma_cam"])
sigma_cam_mean = (sigma_cam**2).mean(dim = "sigma_cam_dim_1")**0.5
sigma_cam_mean_stats = []
for i in range(sigma_cam_mean.sizes["sigma_cam_dim_0"]):
    arr = sigma_cam_mean.sel(sigma_cam_dim_0 = i).values.flatten()
    sigma_cam_mean_stats.append({
        "mean": arr.mean(),
        "sd": arr.std(),
        "hdi_2.5%": np.percentile(arr, 2.5),
        "hdi_97.5%": np.percentile(arr, 97.5)
        })
extra = pd.DataFrame(sigma_cam_mean_stats, index = [
    f"sigma_cam_{i}_mean" for i in range(len(sigma_cam_mean_stats))])
summary = pd.concat([summary, extra])

# Ajout de sigma_ali
sigma_ali = az.extract(trace, var_names = ["sigma_ali"])
sigma_ali_mean = (sigma_ali**2).mean(dim = "sigma_ali_dim_2")**0.5
sigma_ali_mean_stats = []
for i in range(sigma_ali_mean.sizes["sigma_ali_dim_0"]):
    for j in range(sigma_ali_mean.sizes["sigma_ali_dim_1"]):
        arr = sigma_ali_mean.sel(sigma_ali_dim_0 = i,
                                 sigma_ali_dim_1 = j).values.flatten()
        sigma_ali_mean_stats.append({
            "mean": arr.mean(),
            "sd": arr.std(),
            "hdi_2.5%": np.percentile(arr, 2.5),
            "hdi_97.5%": np.percentile(arr, 97.5)
        })

extra = pd.DataFrame(
    sigma_ali_mean_stats,
    index=[f"sigma_ali_{i}_{j}_mean" for i in range(
        sigma_ali_mean.sizes["sigma_ali_dim_0"])
           for j in range(sigma_ali_mean.sizes["sigma_ali_dim_1"])]
)
summary = pd.concat([summary, extra])

display(summary)

Régression avec bande de crédibilité

In [ ]:
# Extraction des postérieurs
a_samples = trace.posterior["a"].values.flatten()
b_samples = trace.posterior["b"].values.flatten()

x_vals = np.linspace(X_obs.min(), X_obs.max(), 100)

# Prédictions postérieures
y_preds = np.array([a + b * x_vals for a, b in zip(a_samples, b_samples)])
y_mean = y_preds.mean(axis = 0)
y_hdi_low, y_hdi_high = np.percentile(y_preds, [2.5, 97.5], axis = 0)

plt.figure(figsize=(12, 8))
plt.fill_between(x_vals, y_hdi_low, y_hdi_high, color = "gray",
                 alpha = 0.3, label="Bande de crédibilité 95%")
plt.plot(x_vals, y_mean, color = "red", label = "Régression bayésienne",
         linewidth = 1, zorder = 1)
plt.errorbar(X_obs, Y_obs, xerr = 4 * uX, yerr = 4 * uY, fmt = 'none',
             label = "Données (incertitude x 4)", color = "black",
             elinewidth = 1)
for i, mat in enumerate(Assigned_Values["Matériau"]):
    plt.scatter(X_obs[i], Y_obs[i], label = mat, s = 10, zorder = 2)
plt.xlabel("X")
plt.ylabel("Y", rotation = 0, labelpad = 50, ha = 'left')
plt.legend()
plt.tight_layout()
plt.show()

Tracés des lois a posteriori

In [ ]:
plt.figure()
plt.hist(a_samples, bins = 40, color = "skyblue", edgecolor = "k")
plt.title("Posterior de a")
plt.xlabel("a")
plt.ylabel("Densité")
plt.show()

plt.hist(b_samples, bins = 40, color = "salmon", edgecolor = "k")
plt.title("Posterior de b")
plt.xlabel("b")
plt.ylabel("Densité")
plt.show()

Tracé de la loi a priori et de celle a posteriori

In [ ]:
x_a = np.linspace(a_samples.min() - 1, a_samples.max() + 1, 300)
x_b = np.linspace(b_samples.min() - 1, b_samples.max() + 1, 300)

plt.figure(figsize = (10, 4))
plt.subplot(1, 2, 1)
plt.hist(a_samples, bins = 40, color = "skyblue", edgecolor = "k",
         density = True, alpha = 0.7, label = "Posterior")
plt.plot(x_a, st.norm.pdf(x_a, loc = 0, scale = 316), 'k--',
         label = "Prior théorique")  # prior de a
plt.title("Posterior de a")
plt.xlabel("a")
plt.ylabel("Densité")
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(b_samples, bins=40, color = "salmon", edgecolor = "k",
         density = True, alpha = 0.7, label = "Posterior")
plt.plot(x_b, st.norm.pdf(x_b, loc = 1, scale = 316), 'k--',
         label = "Prior théorique")  # prior de b
plt.title("Posterior de b")
plt.xlabel("b")
plt.ylabel("Densité")
plt.legend()
plt.tight_layout()
plt.show()